<a href="https://colab.research.google.com/github/liuchiente/code_from_colab_and_kaggle/blob/main/%E7%B8%A3%E5%B8%82%E4%BA%BA%E5%8F%A3%E8%AE%8A%E5%8C%96%E6%83%85%E5%BD%A2%E6%94%B6%E9%9B%86%E5%8F%8A%E6%95%B4%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備環境
1.   掛載硬碟
2.   安裝套件
3.   引入函式庫






In [ ]:
#掛載硬碟
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#切換工作資料夾
import os
if os.path.isdir('/content/drive/My Drive/Colab Notebooks//AnalysisOfPopulation/') is False:
  os.mkdir('/content/drive/My Drive/Colab Notebooks//AnalysisOfPopulation/')
os.chdir('/content/drive/My Drive/Colab Notebooks/AnalysisOfPopulation/')

In [ ]:
#安裝額外套件,在讀取副檔名為xls的excel檔案會用到,安裝完需要重新啟動環境
!pip install xlrd==1.2.0

     |████████████████████████████████| 103 kB 16.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

In [ ]:
#引入函式庫
import datetime
import requests

import numpy as np
import pandas as pd
from numpy.ma.core import flatten_mask

import matplotlib as mpl
import matplotlib.pyplot as plt 
from matplotlib.font_manager import fontManager
import matplotlib.dates as dates
import matplotlib.ticker as ticker

# 資料處理
1.   下載資料
2.   資料清洗



In [ ]:
#下載縣市政府公開資料,內包含人口遷出、遷入等資訊
start_year=2014
end_year=2022
start_month=1
end_month=2

ym_start= 12*start_year + start_month - 1
ym_end= 12*end_year + end_month - 1
for ym in range( ym_start, ym_end ):
    y, m = divmod( ym, 12 )
    yc=(str)(y-1911)
    mc=(str)(m+1).zfill(2)
    fname=yc+mc+'01.xlsx'
    url = 'https://mlhr.miaoli.gov.tw/xls/'+yc+'/'+fname

    r = requests.get(url, allow_redirects=True)
    if r.status_code == 200:
      open(fname, 'wb').write(r.content)
    else:
      fname=yc+mc+'01.xls'
      url = 'https://mlhr.miaoli.gov.tw/xls/'+yc+'/'+fname
       r = requests.get(url, allow_redirects=True)
       open(fname, 'wb').write(r.content)


In [ ]:
#資料清洗,因為公開資料有制式格式,為了方便處理,轉換成方便使用格式
start_year=2016
end_year=2022
start_month=12
end_month=3

ym_start= 12*start_year + start_month - 1
ym_end= 12*end_year + end_month - 1

#資料轉換
for ym in range( ym_start, ym_end ):
    y, m = divmod( ym, 12 )
    yc=(str)(y-1911)
    mc=(str)(m+1).zfill(2)
    fname=yc+mc+'01.xlsx'
    #讀入資料
    print(fname)
    dfn = pd.read_excel(fname)
    #找出可用的Header
    dfn_h=dfn[4:5].values.tolist()
    #重新整理資料
    dfn_c=dfn[4:-1].fillna('-')
    dfn_c.columns=dfn_h
    #存成csv,不紀錄索引
    dfn_c.to_csv(yc+mc+'01.csv',index=0)

# 資料分析
1.   指定篩選條件,進行篩選
2.   繪出統計結果



In [ ]:
#指定篩選條件
target='平安里'
start='201801'
#指定統計起訖,年、月
start_year=2014
end_year=2022
start_month=2
end_month=3

In [ ]:
#統計歷年資料
ym_start= 12*start_year + start_month - 1
ym_end= 12*end_year + end_month - 1
data_list=[]
for ym in range( ym_start, ym_end ):
    y, m = divmod( ym, 12 )
    yc=(str)(y-1911)
    yo=(str)(y)
    mc=(str)(m+1).zfill(2)
    fname=yc+mc+'01.csv'
    dfc=pd.read_csv(fname)
    f1=dfc['區域別'].str.match(target) & dfc['性別'].str.match('計')
    f2=dfc['區域別'].str.match(target) & dfc['性別'].str.match('男')
    f3=dfc['區域別'].str.match(target) & dfc['性別'].str.match('女')

    time=yc+mc
    time_c=yo+mc

    male_in=dfc[f2]['遷出人數_合計'].fillna(0).values[0]
    male_out=dfc[f2]['遷出人數_合計'].fillna(0).values[0]
    female_in=dfc[f3]['遷出人數_合計'].fillna(0).values[0]
    female_out=dfc[f3]['遷出人數_合計'].fillna(0).values[0]
    total_in=dfc[f1]['遷入人數_合計'].fillna(0).values[0]
    total_out=dfc[f1]['遷出人數_合計'].fillna(0).values[0]
    data_list.append((time,time_c,male_in,female_in,total_in,male_out,female_out,total_out))

dna=pd.DataFrame(data_list,columns = ['月份' ,'月份(西元)' , '遷入人數_男', '遷入人數_女', '遷入人數_合計', '遷出人數_男', '遷出人數_女', '遷出人數_合計'])

In [ ]:
#設定顯示所有欄位
pd.set_option("display.max_columns", None)

In [ ]:
#可以看一下統計資料是不是如預期
dna.head(100)

,月份,月份(西元),遷入人數_男,遷入人數_女,遷入人數_合計,遷出人數_男,遷出人數_女,遷出人數_合計
0,10302,201402,36,33,57,36,33,69
1,10303,201403,36,45,74,36,45,81
2,10304,201404,45,55,93,45,55,100
3,10305,201405,51,59,91,51,59,110
4,10306,201406,37,62,93,37,62,99
...,...,...,...,...,...,...,...,...
92,11010,202110,39,41,62,39,41,80
93,11011,202111,25,48,41,25,48,73
94,11012,202112,47,44,53,47,44,91
95,11101,202201,44,50,55,44,50,94


In [ ]:
#繪出統計結果
#設定字型,可以避免matplot無法顯示中文
fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

In [ ]:
#折線圖
dnp=dna
plt.figure(figsize=(35,10))
plt.title(target+"人口變化")
plt.plot(dnp[f1]['月份(西元)'],dnp[f1]['遷入人數_合計'].apply(float),label="遷入人數_合計",linewidth=3)
plt.plot(dnp[f1]['月份(西元)'],dnp[f1]['遷出人數_合計'].apply(float),label="遷出人數_合計",linewidth=3,linestyle='dotted')
plt.xlabel("月份(西元)")
plt.ylabel("人數")
plt.grid(True)
plt.legend(loc = 0, prop={'size': "x-large"})
plt.xticks(rotation=45)

#設定統計區間
axes = plt.gca()
axes.xaxis.set_major_locator(ticker.MultipleLocator(5))

plt.show()

In [ ]:
#長條圖
dnp=dna.head(100)
f1=dnp['月份(西元)']>=start
plt.figure(figsize=(35,10))
plt.title(target+"人口變化")
plt.bar(dnp[f1]['月份(西元)'],dnp[f1]['遷入人數_合計'].apply(float),label="遷入人數_合計")
plt.bar(dnp[f1]['月份(西元)'],dnp[f1]['遷出人數_合計'].apply(float),label="遷出人數_合計")

plt.xlabel("月份")
plt.ylabel("人數")
plt.grid(True)
plt.legend(loc = 0, prop={'size': "x-large"})
plt.xticks(rotation=45)

axes = plt.gca()
axes.xaxis.set_major_locator(ticker.MultipleLocator(5))

plt.show()